In [10]:
import pandas as pd
import re

# Load data
df = pd.read_excel("Webscraping result.xlsx", sheet_name='in')
print(df)

                                           product_title   product_price  \
0            2025 Honda WR-V 1.5 RS SUV - (BEST DEAL SP)  Rp 287.000.000   
1             2025 Honda WR-V 1.5 E SUV - (BEST DEAL SP)  Rp 267.770.000   
2           2021 Honda City 1.5 RS Hatchback - KM 36RIBU  Rp 230.000.000   
3             2025 Honda WR-V 1.5 E SUV - (BEST DEAL SP)  Rp 266.700.000   
4      2019 Honda Brio 1.2 Satya E Hatchback - , MOBI...  Rp 110.000.000   
...                                                  ...             ...   
14364  2023 Honda CR-V 1.5 Turbo Prestige SUV - 1,5 P...  Rp 669.600.000   
14365        2023 Honda HR-V 1.5 Turbo RS SUV - RS Turbo  Rp 529.900.000   
14366  2022 Honda Brio 1.2 Satya E Hatchback - 1,2 E ...  Rp 188.000.000   
14367        2023 Honda HR-V 1.5 Turbo RS SUV - RS Turbo  Rp 527.000.000   
14368                    2023 Honda BR-V 1.5 E SUV - 1,5  Rp 297.000.000   

                                     product_description       brand  
0      0 - 5K KM

In [11]:
# Safely convert product_price to numeric by removing non-digit characters
df['product_price'] = df['product_price'].str.replace(r'[^\d]', '', regex=True)
df['product_price'] = df['product_price'].astype(float).astype('Int64')

In [12]:
df['brand_model'] = df['product_title'].str.extract(r'\d{4}\s+(\w+\s+[\w\-]+)')
df['year'] = df['product_title'].str.extract(r'(\d{4})').astype(float)
df['transmission'] = df['product_description'].str.extract(r'(Automatic|Manual)', flags=re.IGNORECASE)

In [13]:
df['engine_capacity_liters'] = df['product_title'].str.extract(r'\b(\d\.\d)\b')

In [14]:
df['engine_capacity_cc'] = (df['engine_capacity_liters'].astype(float) * 1000).astype(int)

In [15]:
def extract_body_feature(car_text):
    # Regular expression to match the body-feature (everything after the style part)
    pattern = r"^\d{4} Honda [A-Za-z0-9\-]+ \d+\.\d(.*)"
    
    match = re.match(pattern, car_text)
    
    if match:
        body_feature = match.group(1).strip()  # Everything after the style part
        return body_feature
    else:
        return None  # If pattern does not match

# Apply the function to each row and create a new column for Body-Feature
df['features'] = df['product_title'].apply(extract_body_feature)

In [16]:
def extract_max_distance(text):
    if pd.isna(text):
        return None
    
    text = str(text)

    # Case 1: Range format like "0 - 25K KM"
    range_match = re.search(r'(\d+)\s*-\s*(\d+)(K)?\s*KM', text, re.IGNORECASE)
    if range_match:
        max_val = int(range_match.group(2)) * 1000 if range_match.group(3) else int(range_match.group(2))
        return max_val

    # Case 2: Single value format like "6000 KM" or "25K KM"
    single_match = re.search(r'(\d+)(K)?\s*KM', text, re.IGNORECASE)
    if single_match:
        return int(single_match.group(1)) * 1000 if single_match.group(2) else int(single_match.group(1))

    return None

# Apply the function to create new column
df['distance_travelled_KM'] = df['product_description'].apply(extract_max_distance)

In [17]:
df.isnull().sum()

product_title               0
product_price               0
product_description       542
brand                       0
brand_model                 0
year                        0
transmission              542
engine_capacity_liters      0
engine_capacity_cc          0
features                    0
distance_travelled_KM     542
dtype: int64

In [18]:
df.dropna(inplace=True)
df[["brand_model", "year",  "transmission","engine_capacity_cc", "distance_travelled_KM","features", "product_price"]].to_excel("cars_price.xlsx",index=False)